In [4]:
import os
import pandas as pd
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.models.word2vec import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import neighbors
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import nltk
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


from nltk.corpus import stopwords
import multiprocessing

cores = multiprocessing.cpu_count()
doc2VecModel = "doc2Vec_model.sav"
LRdoc2VecModel = "LR_doc2Vec_model.sav"

path1 = "../input/ps-dataset/powershell_benign_dataset/"
path2 = "../input/ps-dataset/malicious_pure"
#path3 = r"D:\Test Scripts"
labels = []
text = []
testText = []
newText = []
#new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
fName= []
regex = r':\s*\n+'
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

def test_bowser(path):
    for filename in os.listdir(path):
        fName.append(filename)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
            t_content = f.read()
            t_content = t_content.lower()
            t_content = t_content.replace("(", "")
            t_content = t_content.replace(")", "")
            t_content = t_content.replace("[", "")
            t_content = t_content.replace("]", "")
            t_content = t_content.replace("$", "")
            t_content = t_content.replace("-", "")
            t_content = t_content.replace("=", "")
            t_content = t_content.replace("", "")
            t_content = t_content.replace("{", "")
            t_content = t_content.replace("}", "")
            t_content = t_content.replace(".", "")
            t_content = t_content.replace("_", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace(":", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace("/", "")
            t_content = t_content.replace("@", "")
            t_content = t_content.replace("?", "")
            t_content = t_content.replace(",", "")
            t_content = t_content.replace(">", "")
            t_content = t_content.replace("<", "")
            t_content = t_content.replace(";", "")
            t_content = t_content.replace("\\", "")
            t_content = t_content.replace("+", "")
            t_content = t_content.replace("\"","")
            t_content = t_content.replace("\'","")
            t_content = t_content.replace("`", "")
            t_content = t_content.replace("*", "")
            t_content = " ".join(t_content.split())
            newText.append(t_content)

def bowser(path, label):
    for filename in os.listdir(path):
        labels.append(label)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
                    content = f.read()
                    content = content.lower()
                    content = content.replace("(", "")
                    content = content.replace(")", "")
                    content = content.replace("[", "")
                    content = content.replace("]", "")
                    content = content.replace("$", "")
                    content = content.replace("-", "")
                    content = content.replace("=", "")
                    content = content.replace("", "")
                    content = content.replace("{", "")
                    content = content.replace("}", "")
                    content = content.replace(".", "")
                    content = content.replace("_", "")
                    content = content.replace("|", "")
                    content = content.replace(":", "")
                    content = content.replace("|", "")
                    content = content.replace("/", "")
                    content = content.replace("@", "")
                    content = content.replace("?", "")
                    content = content.replace(",", "")
                    content = content.replace(">", "")
                    content = content.replace("<", "")
                    content = content.replace(";", "")
                    content = content.replace("\\", "")
                    content = content.replace("+", "")
                    content = content.replace("\"", "")
                    content = content.replace("\'", "")
                    content = content.replace("`", "")
                    content = content.replace("*", "")
                    content = " ".join(content.split())
                    testText.append(content)


#test_bowser(path3)
bowser(path1,"Benign")
bowser(path2,"Malicious")
#print(testText[6])
numFiles = len(testText)

df = pd.DataFrame(testText, index=range(numFiles), columns=["Powershell Code"])
df['Labels'] = labels
#df.to_csv('tester.csv')
print(df)

train, test = train_test_split(df, test_size=0.3, random_state=42)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

#Saving doc2vec model
pickle.dump(model_dbow, open(doc2VecModel, 'wb'))


y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

# logreg = LogisticRegression(n_jobs=1, C=1e5)
# logreg.fit(X_train, y_train)

# #Saving LR model
# pickle.dump(logreg, open(LRdoc2VecModel, 'wb'))

# y_pred = logreg.predict(X_test)

# print(classification_report(y_test,y_pred))

# #print(classification)
# #print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
# #print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))


In [7]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [17]:
for x in range train['Powershell Code']:
    words = tokenize_text(str(train['Powershell Code']))

In [8]:
sentences = train['Powershell Code']
w2v_model.build_vocab(sentences, progress_per=10000)


In [11]:

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

